In [2]:
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
import glob
import os

In [6]:
txt_file_path = '../../aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)
raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Ghost in the mirror', "I knew your face once, but now it's unclear", "And I can't feel my body now"]


In [7]:
# 데이터 정제: preprocess_sentence() 활용
# 토큰 개수 15개 넘지 않게
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제

    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r'[" "]+', " ", sentence)
    # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)

    sentence = sentence.strip()

    # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    sentence = '<start> ' + sentence + ' <end>'

    return sentence

In [8]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue

        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> ghost in the mirror <end>',
 '<start> i knew your face once , but now it s unclear <end>',
 '<start> and i can t feel my body now <end>',
 '<start> i m separate from here and now a drug and a dream <end>',
 '<start> we lost connection , oh come back to me <end>',
 '<start> so i can feel alive again <end>',
 '<start> soul and body try to mend it s pulling me apart , this time <end>',
 '<start> everything is never ending <end>',
 '<start> slipped into a peril that <end>',
 '<start> i ll never understand <end>']

In [9]:
# 토큰화하기
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2 1299   14 ...    0    0    0]
 [   2    5  361 ...    3    0    0]
 [   2    8    5 ...    0    0    0]
 ...
 [   5   22  713 ... 4178    4    3]
 [   5   22  713 ... 4178    4    3]
 [   5   22  713 ... 4178   20    3]] <keras_preprocessing.text.Tokenizer object at 0x7f5b5669e490>


In [10]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

In [11]:
enc_train, enc_val, dec_train, dec_val = train_test_split( src_input, tgt_input, random_state=42,  test_size = 0.2, shuffle = True)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


In [13]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

tr_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
tr_dataset = tr_dataset.batch(BATCH_SIZE, drop_remainder=True)

vl_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
vl_dataset = vl_dataset.batch(BATCH_SIZE, drop_remainder=True)

In [20]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.drop = tf.keras.layers.Dropout(0.3)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.drop(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 2048
hidden_size = 1024
model = None
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [21]:
for src_sample, tgt_sample in tr_dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 7001), dtype=float32, numpy=
array([[[-3.4801276e-05, -4.6270812e-04, -1.8236437e-04, ...,
         -1.4284156e-04, -1.2741965e-04,  3.0782074e-04],
        [-2.2415402e-04, -8.8467076e-04, -7.1863108e-04, ...,
         -2.1394994e-04, -7.0714648e-04,  8.3726953e-04],
        [ 1.3037609e-04, -1.5497439e-03, -9.1975136e-04, ...,
         -7.6895120e-04, -1.1104950e-03,  1.5279602e-03],
        ...,
        [-1.2070598e-03, -2.8885554e-05,  2.4759751e-03, ...,
          1.4667608e-03, -1.4700283e-03,  1.6560509e-03],
        [-9.8437199e-04,  3.6709316e-04,  2.6245208e-03, ...,
          1.6193957e-03, -1.5501054e-03,  2.7231756e-04],
        [-5.8745820e-04,  7.4081431e-04,  2.5208939e-03, ...,
          2.0773930e-03, -1.7171977e-03,  5.5617060e-05]],

       [[-3.4801276e-05, -4.6270812e-04, -1.8236437e-04, ...,
         -1.4284156e-04, -1.2741965e-04,  3.0782074e-04],
        [ 2.8307055e-04, -1.0416297e-03, -3.6664002e-04, ...,
         -3.6434125e-04,  

In [22]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  14338048  
_________________________________________________________________
lstm_2 (LSTM)                multiple                  12587008  
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  7176025   
Total params: 42,493,785
Trainable params: 42,493,785
Non-trainable params: 0
_________________________________________________________________


In [23]:
optimizer = tf.keras.optimizers.Adam()
#Loss
# 모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! 

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
history = model.fit(enc_train, dec_train, 
          validation_data = (enc_val, dec_val), epochs=8)

Epoch 1/8
4394/4394 [==============================] - 441s 100ms/step - loss: 2.9953 - val_loss: 2.7187
Epoch 2/8
4394/4394 [==============================] - 418s 95ms/step - loss: 2.5327 - val_loss: 2.5185
Epoch 3/8
4394/4394 [==============================] - 418s 95ms/step - loss: 2.2636 - val_loss: 2.4231
Epoch 4/8
4394/4394 [==============================] - 419s 95ms/step - loss: 2.0687 - val_loss: 2.3752
Epoch 5/8
4394/4394 [==============================] - 419s 95ms/step - loss: 1.9260 - val_loss: 2.3544
Epoch 6/8
4394/4394 [==============================] - 418s 95ms/step - loss: 1.8218 - val_loss: 2.3517
Epoch 7/8
4394/4394 [==============================] - 419s 95ms/step - loss: 1.7421 - val_loss: 2.3554
Epoch 8/8
4394/4394 [==============================] - 419s 95ms/step - loss: 1.6822 - val_loss: 2.3621


In [29]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다.
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다.
        # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다.
        predict_word = tf.argmax(tf.nn.softmax(
            predict, axis=-1), axis=-1)[:, -1]

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다.
        test_tensor = tf.concat(
            [test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token:
            break
        if test_tensor.shape[1] >= max_len:
            break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다.

    for word_index in test_tensor[0].numpy():
            generated += (tokenizer.index_word[word_index] + " ")

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [30]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so <end> '

결론:   
- 인공지능으로 작사를 할 수 있다는게 흥미로웠다.
- 컴퓨터 성능이 더 발전한 미래에는 훨씬 많고 복잡한 텍스트 데이터로도 손쉽게 구현이 쉬워질 것 같다.
- 지금은 개인 컴퓨터로 더 많고 복잡한 데이터를 처리하기에는 한계가 있는 것 같다.
